In [1]:
# Import libraries
import datetime as dt
import pandas as pd
import re
import numpy as np
import nltk
import regex as reg
import matplotlib.pyplot as plt

from twitterscraper import query_tweets
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from bs4 import BeautifulSoup



pd.set_option('display.max_columns', 500)

INFO: {'User-Agent': 'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}


## DATA COLLECTION

## KINCADE FIRE

In [2]:
# Set defaults/options for the twitter scraper. The date will change for each disaster.
lang= 'english'
kincade_begin_date = dt.date(2019,10,23)
kincade_end_date = dt.date(2019,11,15)
limit = 30000

In [3]:
# Pull tweets with twitter scraper
tweets_kincade = query_tweets('kincadefire', begindate=kincade_begin_date, enddate=kincade_end_date,
                     limit=limit, lang=lang)  # October 23 - November 15, 2019

INFO: queries: ['kincadefire since:2019-10-23 until:2019-10-24', 'kincadefire since:2019-10-24 until:2019-10-25', 'kincadefire since:2019-10-25 until:2019-10-26', 'kincadefire since:2019-10-26 until:2019-10-27', 'kincadefire since:2019-10-27 until:2019-10-28', 'kincadefire since:2019-10-28 until:2019-10-29', 'kincadefire since:2019-10-29 until:2019-10-31', 'kincadefire since:2019-10-31 until:2019-11-01', 'kincadefire since:2019-11-01 until:2019-11-02', 'kincadefire since:2019-11-02 until:2019-11-03', 'kincadefire since:2019-11-03 until:2019-11-04', 'kincadefire since:2019-11-04 until:2019-11-05', 'kincadefire since:2019-11-05 until:2019-11-06', 'kincadefire since:2019-11-06 until:2019-11-08', 'kincadefire since:2019-11-08 until:2019-11-09', 'kincadefire since:2019-11-09 until:2019-11-10', 'kincadefire since:2019-11-10 until:2019-11-11', 'kincadefire since:2019-11-11 until:2019-11-12', 'kincadefire since:2019-11-12 until:2019-11-13', 'kincadefire since:2019-11-13 until:2019-11-15']
INFO

INFO: Using proxy 212.90.168.150:52589
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1190049645721874438-1190055759079337984&q=kincadefire%20since%3A2019-10-31%20until%3A2019-11-01&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1188604670811373570-1188606196514619393&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 212.90.168.150:52589
INFO: Using proxy 212.90.168.150:52589
INFO: Using proxy 212.90.168.150:52589
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1191819631578664960-1191866536970964993&q=kincadefire%20sin

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLT9mcXbhyEWgIC77abd5IchEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-03%20until%3A2019-11-04&l=english
INFO: Using proxy 94.130.125.220:1080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgLW9wZmZ_CAWgMCm1fHumfwgEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-25%20until%3A2019-10-26&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEgKjh0eni_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 94.130.125.220:1080
INFO: 

INFO: Retrying... (Attempts left: 41)
INFO: Using proxy 27.2.7.59:52148
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKOt9dzokCEWgMCgjfelh5IhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-11%20until%3A2019-11-12&l=english
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=kincadefire%20since%3A2019-10-23%20until%3A2019-10-24&l=english
INFO: Using proxy 63.249.67.70:53281
INFO: Using proxy 46.180.156.230:35808
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKel3LH8iCEWgIC3-bD2iYkhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-04%20until%3A2019-11-05&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&inc

INFO: Using proxy 46.180.156.230:35808
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKaRjei6_SAWgICj8djevv0gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-26%20until%3A2019-10-27&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaKwLyBq46djiEWgMC6yaOFu48hEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11-09%20until%3A2019-11-10&l=english
INFO: Using proxy 176.36.164.240:48039
INFO: Using proxy 176.36.164.240:48039
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCgKfp7PSNhSEWgMCiye3ymoUhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-11

...

INFO: Retrying... (Attempts left: 4)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwKOFxc-5_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 151.232.72.12:80
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe1utfagSEWgMC8yfbx0YIhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-29%20until%3A2019-10-31&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 8

INFO: Retrying... (Attempts left: 3)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwKOFxc-5_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 116.254.100.165:46675
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaEwKON1qe6-iAWgICmyY6O9fogEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-24%20until%3A2019-10-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", l

INFO: Retrying... (Attempts left: 2)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaCwKOFxc-5_iAWgoCokbnC4_4gEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-27%20until%3A2019-10-28&l=english
INFO: Using proxy 159.8.114.37:8123
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe1utfagSEWgMC8yfbx0YIhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-29%20until%3A2019-10-31&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 

INFO: Retrying... (Attempts left: 1)
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwKe1utfagSEWgMC8yfbx0YIhEjUAFQAlAFUAFQAA&q=kincadefire%20since%3A2019-10-29%20until%3A2019-10-31&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/

INFO: Got 1466 tweets for kincadefire%20since%3A2019-10-25%20until%3A2019-10-26.
INFO: Got 12692 tweets (1466 new).


In [4]:
# Create dataframe of tweets
df_kincade5=pd.DataFrame(t.__dict__ for t in tweets_kincade)

In [5]:
# Display the number of columns
len(df_kincade5.columns)

21

In [6]:
# Remove hashtags and non-alphanumeric characters
df_kincade5['hashtags'] = df_kincade5['hashtags'].astype(str)
df_kincade5['hashtags'] = df_kincade5.hashtags.str.replace('[^a-zA-Z0-9]','')

In [7]:
df_kincade5['img_urls'] = df_kincade5['img_urls'].astype(str)
df_kincade5['img_urls'] = df_kincade5['img_urls'].str.strip('[]').astype(str)

In [8]:
df_kincade5['links'] = df_kincade5['links'].astype(str)
df_kincade5['links'] = df_kincade5['links'].str.strip('[]').astype(str)

In [9]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data/Fire'

[Errno 2] No such file or directory: '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data/Fire'
/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/code


In [10]:
# Change name of variable for export to folder
tweets_kincadefire =df_kincade5

In [11]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [12]:
# Export dataframe as csv file.
tweets_kincadefire.to_csv('../tweets_kincadefire.csv')

In [13]:
# List value counts for the unique values in the hashtag column
df_kincade5['hashtags'].value_counts().head(3)

KincadeFire    4987
kincadefire     685
Kincadefire     178
Name: hashtags, dtype: int64

## CARR FIRE

In [14]:
# Set dates for the next twitter scrape.
carr_begin_date = dt.date(2018,7,23)
carr_end_date = dt.date(2018,9,7)  # Jul 23, 2018 – Aug 30, 2018 add one week - September 7, 2018

In [15]:
# Pull tweets for the Carr Fire
tweets_carr_fire = query_tweets('carr fire', begindate=carr_begin_date, enddate=carr_end_date,
                     limit=limit, lang=lang)

INFO: queries: ['carr fire since:2018-07-23 until:2018-07-25', 'carr fire since:2018-07-25 until:2018-07-27', 'carr fire since:2018-07-27 until:2018-07-29', 'carr fire since:2018-07-29 until:2018-08-01', 'carr fire since:2018-08-01 until:2018-08-03', 'carr fire since:2018-08-03 until:2018-08-05', 'carr fire since:2018-08-05 until:2018-08-08', 'carr fire since:2018-08-08 until:2018-08-10', 'carr fire since:2018-08-10 until:2018-08-12', 'carr fire since:2018-08-12 until:2018-08-15', 'carr fire since:2018-08-15 until:2018-08-17', 'carr fire since:2018-08-17 until:2018-08-19', 'carr fire since:2018-08-19 until:2018-08-21', 'carr fire since:2018-08-21 until:2018-08-24', 'carr fire since:2018-08-24 until:2018-08-26', 'carr fire since:2018-08-26 until:2018-08-28', 'carr fire since:2018-08-28 until:2018-08-31', 'carr fire since:2018-08-31 until:2018-09-02', 'carr fire since:2018-09-02 until:2018-09-04', 'carr fire since:2018-09-04 until:2018-09-07']
INFO: Querying carr fire since:2018-07-23 un

INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1035992630524559360-1036040714662998016&q=carr%20fire%20since%3A2018-08-31%20until%3A2018-09-02&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1030941184204648448-1030966846004948992&q=carr%20fire%20since%3A2018-08-17%20until%3A2018-08-19&l=english
INFO: Using proxy 212.90.168.150:52589
INFO: Using proxy 212.90.168.150:52589
INFO: Using proxy 212.90.168.150:52589
INFO: Using proxy 212.90.168.150:52589
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1026974244171788288-1026980691777544193&q=carr%20fire%20sin

INFO: Using proxy 94.130.125.220:1080
INFO: Retrying... (Attempts left: 50)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1021848581915979776-1021903268849414145&q=carr%20fire%20since%3A2018-07-23%20until%3A2018-07-25&l=english
INFO: Using proxy 94.130.125.220:1080
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1024442861897445376-1024444565263007744&q=carr%20fire%20since%3A2018-07-29%20until%3A2018-08-01&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/req

ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1023356840955002880-1023357422633668613&q=carr%20fire%20since%3A2018-07-27%20until%3A2018-07-29&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 355, in raw_decode
    

INFO: Using proxy 94.130.125.220:1080
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1030233011445542913-1030242194865750017&q=carr%20fire%20since%3A2018-08-15%20until%3A2018-08-17&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/dec

...

INFO: Using proxy 87.107.236.198:8080
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1021848581915979776-1021903268849414145&q=carr%20fire%20since%3A2018-07-23%20until%3A2018-07-25&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/dec

INFO: Using proxy 203.190.53.45:53927
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1034186504728326144-1034226427875360768&q=carr%20fire%20since%3A2018-08-26%20until%3A2018-08-28&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/dec

INFO: Using proxy 87.107.236.198:8080
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1028420854114205696-1028430173836738560&q=carr%20fire%20since%3A2018-08-10%20until%3A2018-08-12&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/dec

INFO: Retrying... (Attempts left: 2)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1034186504728326144-1034226427875360768&q=carr%20fire%20since%3A2018-08-26%20until%3A2018-08-28&l=english
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1035286189769744384-1035311746301362176&q=carr%20fire%20since%3A2018-08-28%20until%3A2018-08-31&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, *

INFO: Retrying... (Attempts left: 2)
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1036662067858735104-1036759664111714304&q=carr%20fire%20since%3A2018-09-02%20until%3A2018-09-04&l=english
INFO: Using proxy 165.22.224.205:8118
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1033469762183946240-1033500702323208192&q=carr%20fire%20since%3A2018-08-24%20until%3A2018-08-26&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
   

In [16]:
# Create dataframe
df_carr=pd.DataFrame(t.__dict__ for t in tweets_carr_fire)

In [17]:
# Remove non-alphanumeric values/urls and links.
df_carr['hashtags'] = df_carr['hashtags'].astype(str)
df_carr['hashtags'] = df_carr.hashtags.str.replace('[^a-zA-Z0-9]','')
df_carr['img_urls'] = df_carr['img_urls'].astype(str)
df_carr['img_urls'] = df_carr['img_urls'].str.strip('[]').astype(str)
df_carr['links'] = df_carr['links'].astype(str)
df_carr['links'] = df_carr['links'].str.strip('[]').astype(str)

In [18]:
# Export dataframe to csv.
df_carr.to_csv('../df_carr.csv')

## THOMAS FIRE

In [19]:
# Set values for date for next twitter scrape
thomas_begin_date = dt.date(2017,12,4)
thomas_end_date = dt.date(2018,1,19)  # 12/04/2017 - 01/12/2018 plus one week 1/19/2018

In [20]:
# Scrape tweets for the Thomas fire.
thomas_fire = query_tweets('thomas fire', begindate= thomas_begin_date, enddate= thomas_end_date,
                           limit=limit, lang=lang)

INFO: queries: ['thomas fire since:2017-12-04 until:2017-12-06', 'thomas fire since:2017-12-06 until:2017-12-08', 'thomas fire since:2017-12-08 until:2017-12-10', 'thomas fire since:2017-12-10 until:2017-12-13', 'thomas fire since:2017-12-13 until:2017-12-15', 'thomas fire since:2017-12-15 until:2017-12-17', 'thomas fire since:2017-12-17 until:2017-12-20', 'thomas fire since:2017-12-20 until:2017-12-22', 'thomas fire since:2017-12-22 until:2017-12-24', 'thomas fire since:2017-12-24 until:2017-12-27', 'thomas fire since:2017-12-27 until:2017-12-29', 'thomas fire since:2017-12-29 until:2017-12-31', 'thomas fire since:2017-12-31 until:2018-01-02', 'thomas fire since:2018-01-02 until:2018-01-05', 'thomas fire since:2018-01-05 until:2018-01-07', 'thomas fire since:2018-01-07 until:2018-01-09', 'thomas fire since:2018-01-09 until:2018-01-12', 'thomas fire since:2018-01-12 until:2018-01-14', 'thomas fire since:2018-01-14 until:2018-01-16', 'thomas fire since:2018-01-16 until:2018-01-19']
INFO

INFO: Using proxy 212.90.168.150:52589
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-952308401614110720-952328191405735936&q=thomas%20fire%20since%3A2018-01-12%20until%3A2018-01-14&l=english
INFO: Using proxy 212.90.168.150:52589
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-949754100345815040-949790122010058752&q=thomas%20fire%20since%3A2018-01-05%20until%3A2018-01-07&l=english
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-944714239624740865-944718893179920385&q=thomas%20fire%20since%3A2017-12-22%20until%3A2017-12-24&l=english
INFO: Scraping tweets from http

INFO: Using proxy 94.130.125.220:1080
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-938920349512564736-938921045255491585&q=thomas%20fire%20since%3A2017-12-06%20until%3A2017-12-08&l=english
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-943991086816559105-943993677512286208&q=thomas%20fire%20since%3A2017-12-20%20until%3A2017-12-22&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, 

INFO: Using proxy 212.90.168.150:52589
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-939643759209230336-939645817085870086&q=thomas%20fire%20since%3A2017-12-08%20until%3A2017-12-10&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/de

ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-941456826276593664-941457763334471681&q=thomas%20fire%20since%3A2017-12-13%20until%3A2017-12-15&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 355, in raw_decode
    

...

INFO: Retrying... (Attempts left: 4)
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-939643759209230336-939645817085870086&q=thomas%20fire%20since%3A2017-12-08%20until%3A2017-12-10&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/deco

INFO: Retrying... (Attempts left: 2)
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-951579696852062208-951603980211077120&q=thomas%20fire%20since%3A2018-01-09%20until%3A2018-01-12&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/deco

ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-941456826276593664-941457763334471681&q=thomas%20fire%20since%3A2017-12-13%20until%3A2017-12-15&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 355, in raw_decode
    

ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-949754100345815040-949790122010058752&q=thomas%20fire%20since%3A2018-01-05%20until%3A2018-01-07&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 355, in raw_decode
    

INFO: Retrying... (Attempts left: 2)
ERROR: Failed to parse JSON "Expecting value: line 1 column 1 (char 0)" while requesting "https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-946513100236136448-946527707717873667&q=thomas%20fire%20since%3A2017-12-27%20until%3A2017-12-29&l=english"
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/twitterscraper/query.py", line 99, in query_single_page
    json_resp = response.json()
  File "/opt/anaconda3/lib/python3.7/site-packages/requests/models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "/opt/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/lib/python3.7/json/deco

In [21]:
# Create dataframe
df_thomas=pd.DataFrame(t.__dict__ for t in thomas_fire)

In [22]:
# Remove hashtags/urls and links.
df_thomas['hashtags'] = df_thomas['hashtags'].astype(str)
df_thomas['hashtags'] = df_thomas.hashtags.str.replace('[^a-zA-Z0-9]','')
df_thomas['img_urls'] = df_thomas['img_urls'].astype(str)
df_thomas['img_urls'] = df_thomas['img_urls'].str.strip('[]').astype(str)
df_thomas['links'] = df_thomas['links'].astype(str)
df_thomas['links'] = df_thomas['links'].str.strip('[]').astype(str)

In [23]:
# Export dataframe to csv
df_thomas.to_csv('../df_thomas.csv')

## COMBINE KINCADE, CARR, THOMAS FIRE DATAFRAMES

In [24]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [25]:
# Pull in the csv files
kincade = pd.read_csv('../data/tweets_kincadefire.csv')

In [26]:
carr = pd.read_csv('../data/df_carr.csv')

In [27]:
thomas = pd.read_csv('../data/df_thomas.csv')

In [28]:
# Display number of rows and columns
kincade.shape

(12676, 22)

In [29]:
carr.shape

(13674, 22)

In [30]:
thomas.shape

(14032, 22)

In [31]:
# Create a list containing the three dataframes
features = [kincade, carr, thomas]

In [32]:
# Concatenate the three dataframes.
all_fires =pd.concat(features)


In [33]:
# Confirm the concatenation by displaying rows/columns
all_fires.shape

(40382, 22)

In [34]:
# Export the concatenated dataframe to csv
all_fires.to_csv('../data/all_fires.csv')

## FIRES ALL EXPLORATORY DATA ANALYSIS

In [35]:
# Pull in the csv for floods
floods = pd.read_csv('../data/df_floods.csv')

In [36]:
# Display the first row
floods.head(1)

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,requesting_help,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,disaster
0,ThomasKBullock2,Thomas K. Bullock,8.130000e+17,1.090000e+18,/ThomasKBullock2/status/1091479641859198976,2/1/19 23:33,1549063999,"Lord, I hope not. We don’t need another 500 ye...",0.0,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],NaN,1,0,1,True,True,1.090000e+18,"[{'screen_name': 'antmasiello', 'user_id': '54...",floods


In [37]:
# Pull in the fires csv
firefire = pd.read_csv('../data/all_fires.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,13,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
# Create a variable containing the string 'fire'
disaster = 'fire'

In [39]:
# Create a new column in the dataframe and insert the value 'fire' into each row
firefire['disaster']= disaster

In [40]:
# Remove the unwanted columns
firefire= firefire.drop(['Unnamed: 0'], axis=1)  #, 'Unnamed: 0.1'

In [41]:
# Export the cleaned up dataframe to csv
firefire.to_csv('../data/all_fires.csv')